In [ ]:
from pynq.overlays.base import BaseOverlay 
import time
import threading
from datetime import datetime
import multiprocessing
import socket
import asyncio 
import os
import random
import json

In [2]:
base = BaseOverlay("base.bit") 
btns = base.btns_gpio

In [3]:
%%microblaze base.PMODB 
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

In [4]:
def clear_gpio():
    write_gpio(7, 0)
    write_gpio(6, 0)
    write_gpio(5, 0)
    write_gpio(4, 0)
    write_gpio(3, 0)
    write_gpio(2, 0)
    write_gpio(1, 0)

In [8]:
global cond
global blue
global red
global green
global period
global led
global brightness_data
global temp_data

cond = True
blue = 1
red = 2
green = 3
period = .5
led = red
brightness_data = 0
temp_data = 0

async def pulse_led():
    global led
    global period
    global cond
    print('pulse led')

    write_gpio(blue,0)
    write_gpio(red,0)
    write_gpio(green,0)

    while True:
        period = round((brightness_data + 1) / 1024) ** 2 * .1
        if temp_data < 1365:
            led = blue
        elif temp_data < 2730:
            led = red
        elif temp_data < 4095:
            led = green
        print(period)
        print(led)
        clear_gpio()
        write_gpio(led,1)

        await asyncio.sleep(period)
        write_gpio(led,0)

        await asyncio.sleep(period)



In [12]:
def server(PORT):
    global led
    global period
    global cond
    global temp_data
    global brightness_data
    print('Server Started')
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    HOST = '0.0.0.0' 
    sock.bind((HOST, PORT))
    sock.listen(1)
    conn, addr = sock.accept() #conn is a socket object, why is addr ('192.168.2.1', 59527)
    print('Connected by', addr)
    with conn:
        while True:
            loop = asyncio.new_event_loop()
            loop.create_task(get_data(conn))
            loop.create_task(pulse_led())
            loop.run_forever()
            loop.close() 
            
async def get_data(conn):
    global led
    global period
    global cond
    global temp_data
    global brightness_data
    while True:
        data = conn.recv(1024)
            
        if data:
            if data == 'b/close': 
                print('Closing')
                loop.close()
                sock.close()
            else:
                l_sensor_data = json.loads(data.decode('ASCII'))
                print(l_sensor_data)
                brightness_data = int(l_sensor_data[0])
                temp_data = int(l_sensor_data[1])
                #maybe need to implement an ack
                    
        await asyncio.sleep(4)


     

In [13]:
server(1001)

Server Started
Connected by ('192.168.2.1', 65052)
[3774, 2135]
pulse led
1.6
2
1.6
2
[2463, 2560]
0.4
2
0.4
2
[3164, 902]
0.9
1
0.9
1
0.9
1
[967, 2225]
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
0.1
2
[3491, 1711]
0.9
2
0.9
2
0.9
2
[292, 2924]
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3
0.0
3


KeyboardInterrupt: 

In [ ]:
procs = [] # a future list of all our processes

print('starting p1')
# Launch process1 on CPU0
p1 = multiprocessing.Process(target=server, args=(1001,))
os.system("taskset -p -c {} {}".format(1, p1.pid)) # taskset is an os command to pin the process to a specific CPU
procs.append(p1)


print('Start')
p1.start() # start the process

p1Name = p1.name # get process1 name
                             
p1.join()


print('Process 1 with name, {}, is finished'.format(p1Name))
